<h1>Deep learning project 8DM20 CSMIA</h1>

<h4>Group members:</h4>
O. Akdag - 0842508 <br>
T.P.A. Beishuizen - 0791613 <br>
A.S.A. Eskelinen - 1224333 <br>
J.H.A. Migchielsen - 0495058 <br>
L. van den Wildenberg - 0844697 <br>

In [ ]:
# Import all used packages (unused packages are commented out so far)
import os
from PIL import Image as PIL_image
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as img
from sklearn.feature_extraction import image as sklearn_image
#matplotlib inline
import theano
import lasagne
import time
import random
import math
random.seed(0)

Before every thing can be done, at first the data images have to be read and be made in useable data types.

In [ ]:
# The function that loads the data
def loadData(data_set = 'test', image = '1st_manual'):
    
    # Check for the correct input
    if data_set != 'test' and data_set != 'training':
        raise Exception('Not the right data_set file')
    if image != '1st_manual' and image != '2nd_manual' and image != 'images' and image != 'mask':
        raise Exception('Not the right image file')
    if data_set == 'training' and image == '2nd_manual':
        raise Exception('File not available')
    
    # Project and image path
    project_path = os.getcwd()
    images_path = project_path +  '/8DM20_image_dataset/' + data_set + '/' + image + '/'
    
    images = []
    
    #Open image for image (20 in total for each of them)
    for i in range(1, 21):
        
        # Find correct image number
        image_nr = str(i)
        if data_set == 'training':
            image_nr = str(20 + i)
        elif len(image_nr) == 1:
            image_nr = '0' + image_nr
            
        # Specify path for this image
        if image == '1st_manual':
            image_path = images_path + image_nr + '_manual1.gif'
        elif image == '2nd_manual':
            image_path = images_path + image_nr + '_manual2.gif'
        elif image == 'images':
            image_path = images_path + image_nr + '_' + data_set + '.tif'
        elif image == 'mask':
            image_path = images_path + image_nr + '_' + data_set + '_mask.gif'
        
        # Open and append the image to the image list
        images.append(PIL_image.open(image_path))
        
    return images

#The function that converts the channels in the images from RGB to gray
#and makes matrices from the images
def convertImageToMatrix(images):
    
    image_matrices = []
    
    for image in images:
        image_matrix = np.asarray(image.convert('L'))
        image_matrices.append(image_matrix)
        
    return image_matrices

#The function that prepares the image matrices to the data used for machine learning
def prepareMachineLearningData(image_matrix, output_matrix, mask_matrix, kernel_size, mask_removal = 'pixel'):
    #Check if correct input
    if not (isinstance(image_matrix, np.ndarray) and 
            isinstance(output_matrix, np.ndarray) and 
            isinstance(mask_matrix, np.ndarray)):
        raise Exception("Not all input matrices are numpy matrices")
    
    if mask_removal != 'pixel' and mask_removal != 'patch':
        raise Exception("Unknown mask data removal type")
    
    if not (image_matrix.shape == output_matrix.shape == mask_matrix.shape):
        raise Exception("The images are not the same size")
    
    if np.unique(mask_matrix).shape[0] != 2:
        raise Exception("The mask matrix does not consist of only 2 values")
    
    #Creates a matrix with all possible patches
    all_image_patches = sklearn_image.extract_patches_2d(image_matrix,(kernel_size,kernel_size))
    all_image_patches = np.expand_dims(all_image_patches, axis=1)
    
    if kernel_size % 2 != 0:
         # Creates an array with all output
        mat_red = (kernel_size - 1) / 2
        reduced_output_matrix = output_matrix[ mat_red : -  mat_red,  mat_red : -  mat_red]
        complete_output_array = reduced_output_matrix.reshape(-1)

        new_mask_matrix = mask_matrix.copy()
        
        # Makes some additional mask values zero on the edge of the mask
        if mask_removal == 'patch':
            for i in range(mat_red, mask_matrix.shape[0] -  mat_red + 1):
                for j in range(mat_red, mask_matrix.shape[1] -  mat_red + 1):
                    if 0 in mask_matrix[i - mat_red : i + mat_red + 1, j - mat_red: j + mat_red + 1]:
                        new_mask_matrix[i,j] = 0;
        
        # Creates an array with all mask locations
        reduced_mask_matrix = new_mask_matrix[ mat_red : -  mat_red, mat_red : -  mat_red]
        mask_array = reduced_mask_matrix.reshape(-1)
    
    else:
        # Creates an array with all output
        mat_red = (kernel_size) / 2
        reduced_output_matrix = output_matrix[mat_red - 1: -  mat_red,  mat_red - 1: -  mat_red]
        complete_output_array = reduced_output_matrix.reshape(-1)

        new_mask_matrix = mask_matrix.copy()
        
        # Makes some additional mask values zero on the edge of the mask
        if mask_removal == 'patch':
            for i in range(mat_red - 1, mask_matrix.shape[0] -  mat_red + 1):
                for j in range(mat_red - 1, mask_matrix.shape[1] -  mat_red + 1):
                    if 0 in mask_matrix[i - mat_red + 1 : i + mat_red + 1, j - mat_red + 1: j + mat_red + 1]:
                        new_mask_matrix[i,j] = 0;
                     
        # Creates an array with all mask locations
        reduced_mask_matrix = new_mask_matrix[mat_red - 1: - mat_red, mat_red - 1: - mat_red]
        mask_array = reduced_mask_matrix.reshape(-1) 

    image_patches = []
    output_array = []
    
    # Reduces the number of patches and output to only the mask values
    for i in range(len(mask_array)):
        if mask_array[i] != 0:
            image_patches.append(all_image_patches[i,:,:])
            output_array.append(complete_output_array[i])
        
    
    # Return the image patches and the output array
    return image_patches, output_array

# Prepare multiple images at once
def prepareMultipleImages(image_matrices, output_matrices, mask_matrices, kernel_size = 256, mask_removal = 'pixel'):
    #Check if correct input
    if not (isinstance(image_matrices, list) and 
            isinstance(output_matrices, list) and 
            isinstance(mask_matrices, list)):
        raise Exception("Not all input matrices are numpy matrices")
    
    if not (len(image_matrices) == len(output_matrices) == len(mask_matrices)):
        raise Exception("The images are not the same size")
    
    image_patches = [] 
    output_arrays = []
    
    # Finds the output data per image
    for i in range(len(image_matrices)):
        new_image_patches, new_output_array = prepareMachineLearningData(image_matrices[i], output_matrices[i], mask_matrices[i], 
                                                                         kernel_size = kernel_size, mask_removal = mask_removal)
        image_patches.append(new_image_patches)
        output_arrays.append(new_output_array)
        
        #Print progress for showing time consumption
        print"Progress: {} %".format(100*(i+1)/len(image_matrices)),
              
    return image_patches, output_arrays

def createVesselImage(output_array, mask_matrix, kernel_size):
    #Check if input is correct
    if not isinstance(output_array, list) or not isinstance(mask_matrix, np.ndarray) or not isinstance(kernel_size, int):
        raise Exception("Not the right input variables")
    
    #Create an output_matrix for the output array
    #output_matrix = np.array(mask_matrix)
    output_matrix = np.zeros(mask_matrix.shape)
    output_loc = 0
    
    # Take into account that mask pixels too close to the border are lost due to inability to make patches
    edge_corr = math.ceil(kernel_size / 2) - 1
    
    # Check every pixel within the mask for a vessel pixel
    for i in range(mask_matrix.shape[0] - kernel_size + 1):
        for j in range(mask_matrix.shape[1] - kernel_size + 1):
            if mask_matrix[i + edge_corr, j + edge_corr] == 255:
                output_matrix[i + edge_corr, j + edge_corr] = output_array[output_loc]
                output_loc += 1
                
    return output_matrix 

All images are loaded and immediately made into matrices for further computation

In [ ]:
# All test image lists
test_manual1_images = loadData('test', '1st_manual')
test_manual2_images = loadData('test', '2nd_manual')
test_raw_images = loadData('test', 'images')
test_mask_images = loadData('test', 'mask')

# Making matrices of the test images to work with
test_manual1_matrices = convertImageToMatrix(test_manual1_images)
test_manual2_matrices = convertImageToMatrix(test_manual2_images)
test_raw_matrices = convertImageToMatrix(test_raw_images)
test_mask_matrices = convertImageToMatrix(test_mask_images)

# All training image lists
training_manual1_images = loadData('training', '1st_manual')
training_raw_images = loadData('training', 'images')
training_mask_images = loadData('training', 'mask')

# Making matrices of the training images to work with
training_manual1_matrices = convertImageToMatrix(training_manual1_images)
training_raw_matrices = convertImageToMatrix(training_raw_images)
training_mask_matrices = convertImageToMatrix(training_mask_images)

The matrices are then used for further preprocessing to retrieve the suitable data for machine learning

In [ ]:
#Choose the number of images
nr_images_training = 2
nr_images_test = 1

# Prepares the data for machine learning: X = image_patches, y = output_array
# Both are a list with the patches and output_arrays for multiple images (the number chosen before)
image_patches, output_array = prepareMultipleImages(training_raw_matrices[0:nr_images_training], training_manual1_matrices[0:nr_images_training], 
                                                     training_mask_matrices[0:nr_images_training], 256, mask_removal = 'patch')
test_image_patches, test_output_array = prepareMultipleImages(test_raw_matrices[0:nr_images_test], test_manual1_matrices[0:nr_images_test], 
                                                     test_mask_matrices[0:nr_images_test], 256, mask_removal = 'patch')
print np.shape(image_patches)
print np.shape(test_image_patches)

The following code is just to show how the data set is built up. There are patches of 32 x 32. These values either correspond to a vene pixel or not.

In [ ]:
image_number = 1
pixel = 1 #35645


np.asarray(image_patches[1]).shape

plt.matshow(image_patches[image_number][pixel][0])
plt.show()

print("Should be vene pixel? " + str(output_array[image_number - 1][pixel]) + " (255 is yes, 0 is no)")

For using machine learning. At first training and validation data has to be set up. This is almost done in the previous pre-processing part, however the data set still need to be assigned.

In [ ]:
def train_and_validation_set(image_patches, output_array):
    all_train_patches = []
    all_train_output = []

    for i in range(nr_images_training):
        if i <= (nr_images_training-1)/2 :
            for j in range(len(image_patches[i])):
                all_train_patches.append(image_patches[i][j])
                all_train_output.append(output_array[i][j])
        else:
            valid_patches = image_patches[i]
            valid_output = output_array[i]
    
    return all_train_patches, all_train_output, valid_patches, valid_output

def hot_encoding(all_train_output, valid_output):
    train_hot_output = np.zeros((len(all_train_output),2),dtype=np.int16)

    # Make hot encoding training set
    for i in range(len(train_hot_output)):
        if all_train_output[i] == 0:
            train_hot_output[i,0] = 1
        else:
            train_hot_output[i,1] = 1      

    # Make hot encoding validation set
    valid_hot_output = np.zeros((len(valid_output),2),dtype=np.int16)

    for i in range(len(valid_hot_output)):
        if valid_output[i] == 0:
            valid_hot_output[i,0] = 1
        else:
            valid_hot_output[i,1] = 1
    
    return train_hot_output, valid_hot_output


def test_set(test_image_patches, test_output_array):
    all_test_patches = []
    all_test_output_array = []
    
    for i in range(nr_images_test):
        for j in range(len(test_image_patches[i])):
            all_test_patches.append(test_image_patches[i][j])
            all_test_output_array.append(test_output_array[i][j])
                
    return all_test_patches, all_test_output_array


Important to first make the output array. This is done with hot encoding

In [ ]:
all_train_patches, all_train_output, valid_patches, valid_output = train_and_validation_set(image_patches, output_array)
train_hot_output, valid_hot_output = hot_encoding(all_train_output, valid_output)

all_test_patches, all_test_output_array = test_set(test_image_patches, test_output_array)

print np.shape(all_train_patches)
print np.shape(all_test_patches)

Build the LeNet network

In [ ]:
def buildLeNet(X1):
    inputlayer = lasagne.layers.InputLayer(shape=(None, 1, 32, 32),input_var=X1)    
    print inputlayer.output_shape
    
    layer1 = lasagne.layers.Conv2DLayer(inputlayer, num_filters=64, filter_size=(3,3), nonlinearity=lasagne.nonlinearities.sigmoid, W=lasagne.init.GlorotUniform())
    print layer1.output_shape 
    
    layer2 = lasagne.layers.Conv2DLayer(layer1, num_filters=64, filter_size=(3,3), nonlinearity=lasagne.nonlinearities.sigmoid, W=lasagne.init.GlorotUniform())
    print layer1.output_shape 
    
    layer3 = lasagne.layers.MaxPool2DLayer(layer2, pool_size=(2, 2))
    print layer2.output_shape 
    
    layer4 = lasagne.layers.Conv2DLayer(layer3, num_filters=128, filter_size=(3,3), nonlinearity=lasagne.nonlinearities.sigmoid, W=lasagne.init.GlorotUniform())
    print layer3.output_shape 
    
    layer5 = lasagne.layers.Conv2DLayer(layer4, num_filters=128, filter_size=(3,3), nonlinearity=lasagne.nonlinearities.sigmoid, W=lasagne.init.GlorotUniform())
    print layer1.output_shape 
    
    layer6 = lasagne.layers.MaxPool2DLayer(layer5, pool_size=(2, 2))
    print layer4.output_shape 
    
    layer7 = lasagne.layers.Conv2DLayer(layer6, num_filters=256, filter_size=(3,3), nonlinearity=lasagne.nonlinearities.sigmoid, W=lasagne.init.GlorotUniform())
    print layer1.output_shape 
    
    layer8 = lasagne.layers.Conv2DLayer(layer7, num_filters=256, filter_size=(3,3), nonlinearity=lasagne.nonlinearities.sigmoid, W=lasagne.init.GlorotUniform())
    print layer1.output_shape 
    
    layer9 = lasagne.layers.Conv2DLayer(layer8, num_filters=256, filter_size=(1,1), nonlinearity=lasagne.nonlinearities.sigmoid, W=lasagne.init.GlorotUniform())
    print layer1.output_shape 
    
    layer10 = lasagne.layers.MaxPool2DLayer(layer9, pool_size=(2, 2))
    print layer4.output_shape 
    
    layer11 = lasagne.layers.Conv2DLayer(layer10, num_filters=512, filter_size=(3,3), nonlinearity=lasagne.nonlinearities.sigmoid, W=lasagne.init.GlorotUniform())
    print layer1.output_shape 
    
    layer12 = lasagne.layers.Conv2DLayer(layer11, num_filters=512, filter_size=(3,3), nonlinearity=lasagne.nonlinearities.sigmoid, W=lasagne.init.GlorotUniform())
    print layer1.output_shape 
    
    layer13 = lasagne.layers.Conv2DLayer(layer12, num_filters=512, filter_size=(1,1), nonlinearity=lasagne.nonlinearities.sigmoid, W=lasagne.init.GlorotUniform())
    print layer1.output_shape 
    
    layer14 = lasagne.layers.MaxPool2DLayer(layer13, pool_size=(2, 2))
    print layer4.output_shape 
    
    layer15 = lasagne.layers.Conv2DLayer(layer14, num_filters=512, filter_size=(3,3), nonlinearity=lasagne.nonlinearities.sigmoid, W=lasagne.init.GlorotUniform())
    print layer1.output_shape 
    
    layer16 = lasagne.layers.Conv2DLayer(layer15, num_filters=512, filter_size=(3,3), nonlinearity=lasagne.nonlinearities.sigmoid, W=lasagne.init.GlorotUniform())
    print layer1.output_shape 
    
    layer17 = lasagne.layers.Conv2DLayer(layer16, num_filters=512, filter_size=(1,1), nonlinearity=lasagne.nonlinearities.sigmoid, W=lasagne.init.GlorotUniform())
    print layer1.output_shape 
    
    layer18 = lasagne.layers.MaxPool2DLayer(layer17, pool_size=(2, 2))
    print layer4.output_shape 
    
    layer19 = lasagne.layers.GlobalPoolLayer(layer18, pool_function=theano.tensor.mean)
    print layer19.output_shape
    
    outputlayer = lasagne.layers.DenseLayer(layer19,num_units=2,nonlinearity=lasagne.nonlinearities.sigmoid)     
    print outputlayer.output_shape 
    
    return layer1, layer2, layer3, layer4, layer5, layer6, outputlayer

In [ ]:
X = theano.tensor.tensor4()
Y = theano.tensor.matrix()
layer1, layer2, layer3, layer4, layer5, layer6, layer7, layer8, layer8, layer10, layer11, layer12, layer13, layer14, layer15, layer16, layer17, layer18, layer19, outputlayer = buildLeNet(X)

Defining functions for training, validating and testing the previously made LeNet network

In [ ]:
outputtrain = lasagne.layers.get_output(outputlayer) #function that gets the output from the network defined before.
trainloss = lasagne.objectives.categorical_crossentropy(outputtrain, Y).mean() #function that computes the mean crossentropy between the output and the real labels.
params = lasagne.layers.get_all_params(outputlayer, trainable=True) #function that gets all the parameters (weights) in the network.
updates = lasagne.updates.momentum(trainloss, params, learning_rate=0.001) #function that performs an update of the weights based on the loss.
train = theano.function(inputs=[X, Y], outputs=trainloss, updates=updates, allow_input_downcast=True) #function that does all the above based on training samples X and real labels Y.

validate = theano.function(inputs=[X, Y], outputs=trainloss, allow_input_downcast=True) #function that computes the loss without performing an update

outputtest = lasagne.layers.get_output(outputlayer, deterministic=True) #function that gets the output from the network defined before.
test = theano.function(inputs=[X], outputs=outputtest, allow_input_downcast=True) #function that gets the output based on input X

Then it is time to train the network

In [ ]:
def training_the_network(all_train_output, valid_output, all_train_patches, valid_patches, minibatches = 250, minibatchsize = 100):

    trainingsamples = np.arange(len(all_train_output)) #numbers from 0 until the number of samples
    validsamples = np.arange(len(valid_output))

    losslist = []
    validlosslist = []

    t0 = time.time()

    for i in xrange(minibatches):
        print("Currently at batch %d" % i)

        # Random train sample information. IMPORTANT: Use the hot encoded labels (that's the way the algorithm works)
        random_train_samples = random.sample(trainingsamples, minibatchsize)
        random_train_output = train_hot_output[random_train_samples]
        random_train_patches = np.asarray(all_train_patches)[random_train_samples]

        # Random validation sample information IMPORTANT: Use the hot encoded labels (that's the way the algorithm works)
        random_valid_samples = random.sample(validsamples, minibatchsize)
        random_valid_output = valid_hot_output[random_valid_samples]
        random_valid_patches = np.asarray(valid_patches)[random_valid_samples]

        print(random_train_output.shape)
        print(random_train_patches.shape)

        new_train_loss = train(random_train_patches, random_train_output)
        losslist.append(new_train_loss)

        new_valid_loss = validate(random_valid_patches, random_valid_output)
        validlosslist.append(new_valid_loss)
        #select random training en validation samples and perform training and validation steps here.

    t1 = time.time()
    print 'Training time: {} seconds'.format(t1-t0)
    
    return losslist, validlosslist

In [ ]:
losslist, validlosslist = training_the_network(all_train_output, valid_output, all_train_patches, valid_patches, minibatches = 250, minibatchsize = 100)

In [ ]:
plt.figure()
plt.plot(losslist)
plt.plot(validlosslist)
plt.legend(['Training loss','Validation loss'])
plt.show()

In [ ]:
output_matrix = createVesselImage(output_array[1], training_mask_matrices[1], 32)

plt.matshow(output_matrix)
plt.matshow(training_manual1_matrices[1])
plt.show()

<h4>Evaluation on the test set</h4>

In [ ]:
t0 = time.time()
testing = test(all_test_patches[0:20])
test_set_predictions = np.argmax(testing, axis = 1)
t1 = time.time()
print 'Testing time: {} seconds'.format(t1-t0)

print test_set_predictions
print sum(test_set_predictions)

In [ ]:
def label_test(all_test_output_array):
    label_test_output = np.zeros(len(all_test_output_array),dtype=np.int16)

    for i in range(len(all_test_output_array)):
        if all_test_output_array[i] == 0:
            label_test_output[i] = 0   
        else:
            label_test_output[i] = 1
    
    return label_test_output

In [ ]:
label_test_output = label_test(all_test_output_array)

In [ ]:
TP = np.sum(label_test_output[0:10] == test_set_predictions) 
print 'Accuracy: {}'.format(float(TP)/float(len(all_test_patches[0:10])))
print TP

<h4>What has the network learned?</h4>

In [ ]:
outputlayer1 = lasagne.layers.get_output(layer1) 
outputfeatures = theano.function(inputs=[X], outputs=outputlayer1, allow_input_downcast=True) 

In [ ]:
features = outputfeatures(all_test_patches[1000:58621])
print np.shape(features)

In [ ]:
print features.shape
for i in xrange(6):
    plt.figure()
    plt.imshow(features[1,i],cmap='gray_r',interpolation='none')
    plt.show()

<h4> Visualising the filters </h4>


In [ ]:
weights = lasagne.layers.get_all_param_values(layer1)
filters = weights[0]
biases = weights[1]

In [ ]:
print filters.shape
print biases.shape
for i in xrange(6):
    plt.figure()
    plt.imshow(filters[i,0],cmap='gray_r',interpolation='none')
    plt.show()

<h4> Saving the network </h4>

In [ ]:
def save_weights(filename,network):
    with open(filename, 'wb') as f:
        cPickle.dump(lasagne.layers.get_all_param_values(network), f)
        
def load_weights(filename, network):
    with open(filename, 'rb') as f:
        lasagne.layers.set_all_param_values(network, cPickle.load(f))